In [1]:
import pandas as pd
import xmlrpc.client
import pandas as pd
import numpy as np

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)


# TIEMPOS ESTANDAR

In [2]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name','workcenter_id']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['centro_produccion'] = test['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)


test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)



pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")

# Merge claficacion

test = test.rename(columns={"total_nbr_minimo":"tiempo_estimado_estandard" , "display_name":"maquina_esperada"})

pareto_prod = pareto_prod[["nom_produccion","Clasificacion"]] #Columna realmente requeridas
te_clas = test.merge(pareto_prod, how="left",on="nom_produccion") 

# Descartar productos sin ventas ya que no tienen clasificacion
sin_ventas = te_clas[te_clas["Clasificacion"].isna()] 
te_clas = te_clas[~te_clas["nom_produccion"].isin(sin_ventas["nom_produccion"])]

te_clas = te_clas[["nom_produccion","producto","Clasificacion","centro_produccion","tiempo_estimado_estandard","maquina_esperada"]]

# 🧹 Eliminar filas con tiempo_estimado_estandard = 0 excepto si maquina_esperada == "BOLSA"
te_clas = te_clas[
    ~((te_clas["tiempo_estimado_estandard"] == 0) & (te_clas["maquina_esperada"].str.upper() != "BOLSA"))
]


In [3]:
te_clas[te_clas["producto"].str.contains("DA2547")]

,nom_produccion,producto,Clasificacion,centro_produccion,tiempo_estimado_estandard,maquina_esperada
810,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,CORTE,50.0,ENTUBADORA 3
1225,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,CORTE,40.0,ENTUBADORA 2
1936,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,PLISADO,170.0,PLISADORA 4
2428,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,INYECCION,225.0,HENNECKE
2998,DA2547UHE,DA2547UHE FILTRO AIRE ALTA EFICIENCIA - KODIAK - KENWORTH - FORD,B,EMBALAJE,0.0,BOLSA
3303,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,EMBALAJE,0.0,BOLSA
4026,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,EMBALAJE,40.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE
4622,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,EMBALAJE,30.0,CONTROL DE CALIDAD
5148,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",AAA,EMBALAJE,45.0,EMBALAJE


In [4]:
te_clas["centro_produccion"].unique()

array(['EMBALAJE', 'CORTE', 'PLISADO', 'INYECCION', 'INTERNOS GUATA'],
      dtype=object)

# INFO OP

In [5]:
import xmlrpc.client
import pandas as pd
import re

# --- Configuración de conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- 1️⃣ Órdenes de producción terminadas ---
producciones_done = models.execute_kw(
    db, uid, password,
    "mrp.production", "search_read",
    [[("state", "=", "done")], ["id", "name"]]
)
df_producciones = pd.DataFrame(producciones_done)
ids_done = df_producciones["id"].tolist()

# --- 2️⃣ Work orders asociadas ---
workorders = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line.flab", "search_read",
    [
        [("production_id", "in", ids_done)],
        [
           "workcenter_id","cantidad_fabricada","display_name","operaion","production_id",
           "product_reference" ,"delay","fecha_final"

        ]
    ]
)

df_workorders = pd.DataFrame(workorders)

df_workorders["OP"] = df_workorders["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)


df_workorders["nombre"] = df_workorders["operaion"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)

df_workorders["centro_produccion"] = df_workorders["workcenter_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)

df_workorders["maquina_real"] = df_workorders["display_name"].str.split(" - ", n=1).str[0].str.strip()

df_workorders["nom_produccion1"] = df_workorders["display_name"].str.split(" - ", n=1).str[1]

df_workorders["nom_produccion"] = df_workorders["nom_produccion1"].str.extract(r"\b(DA\d+[A-Za-z]?)\b")


df_workorders = df_workorders.drop(columns=["operaion","workcenter_id","production_id","product_reference",
                                            "nom_produccion1"])

#TARDA 2 min

In [6]:
df_workorders["tiempo_real"] = (df_workorders["delay"] / df_workorders["cantidad_fabricada"]).round(2) #IMPORTANTE DEJAR AQUI

df_workorders[(df_workorders["nom_produccion"]=="DA2547") & (df_workorders["OP"]=="MO12004") ].sort_values(by=["centro_produccion"], ascending=True).head(20)

df_workorders.sample(5)

,delay,display_name,fecha_final,cantidad_fabricada,id,OP,nombre,centro_produccion,maquina_real,nom_produccion,tiempo_real
30552,34578.0,"SOLDADURA - DA8190 FILTRO AIRE CHEVROLET NMR, NPS 2020",2021-11-09 10:11:00,200.0,31591,MO07026,BLANCA DORA CUBIDES SANCHEZ,CORTE,SOLDADURA,DA8190,172.89
47116,9709.0,"MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA2416 FILTRO AIRE CAT, CASE, CLARK, KOBELCO, J.DEERE",2023-02-02 14:54:48,100.0,48570,MO08479,ERIKA YULIEDT UMAÑA SABOGAL,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,DA2416,97.09
59009,4464.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA2366 FILTRO AIRE 1_ JOHN DEERE MOTONIVELADORA,2023-08-16 06:39:40,30.0,60506,MO09380,ERIKA YULIEDT UMAÑA SABOGAL,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,DA2366,148.80
100259,1999.0,"MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA4547 FILTRO AIRE 2_ FORD 7000, KODIAK, IHC.",2025-05-23 11:55:37,350.0,101898,MO12429,ERIKA YULIEDT UMAÑA SABOGAL,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,DA4547,5.71
28065,12060.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA8051 FILTRO AIRE FOTON OLIN BJ1039 5.8 TON.,2021-09-10 15:02:18,600.0,29030,MO06757,LEYDI PAOLA REYES RUBIO,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,DA8051,20.10


In [7]:
df_workorders["centro_produccion"].unique()

array(['CORTE', 'PLISADO', 'INTERNOS GUATA', 'INYECCION', 'EMBALAJE'],
      dtype=object)

# UNION

In [8]:
# 1️⃣ Agrupamos los tiempos estándar por producto, centro y máquina
te_agg = (
    te_clas.groupby(["nom_produccion", "centro_produccion", "maquina_esperada"], as_index=False)
    .agg({"tiempo_estimado_estandard": "sum", "Clasificacion": "first"})
    .rename(columns={"tiempo_estimado_estandard": "tiempo_estandar_total"})
)

# 2️⃣ Primer merge: coincidencia exacta (incluye máquina)
merge_exact = pd.merge(
    df_workorders,
    te_agg,
    left_on=["nom_produccion", "centro_produccion", "maquina_real"],
    right_on=["nom_produccion", "centro_produccion", "maquina_esperada"],
    how="left"
)
merge_exact["tipo_union"] = "exacta"

# 3️⃣ Detectamos los que no encontraron match exacto
faltantes = merge_exact[merge_exact["tiempo_estandar_total"].isna()].copy()

# 4️⃣ Segundo merge: solo por producto y centro (ignora máquina, pero conserva la esperada)
merge_flexible = pd.merge(
    faltantes.drop(columns=["maquina_esperada", "Clasificacion", "tiempo_estandar_total", "tipo_union"]),
    te_clas.groupby(["nom_produccion", "centro_produccion"], as_index=False)
            .agg({
                "tiempo_estimado_estandard": "max",
                "Clasificacion": "first",
                "maquina_esperada": "first"   # ✅ conservar una máquina esperada
            })
            .rename(columns={"tiempo_estimado_estandard": "tiempo_estandar_total"}),
    on=["nom_produccion", "centro_produccion"],
    how="left"
)
merge_flexible["tipo_union"] = "por_centro"


# 5️⃣ Unimos ambos resultados
df_join = pd.concat(
    [merge_exact[~merge_exact["tiempo_estandar_total"].isna()], merge_flexible],
    ignore_index=True
)

# 6️⃣ Calculamos métricas
df_join["desviacion_abs"] = df_join["tiempo_real"] - df_join["tiempo_estandar_total"]
df_join["precision_%"] = (
    (df_join["tiempo_real"] / df_join["tiempo_estandar_total"]) * 100
).replace([float("inf"), -float("inf")], 0)



In [9]:
df_final = df_join[df_join["fecha_final"]>"2023-01-01"]

In [10]:
df_final[(df_final["nom_produccion"]=="DA2547") & (df_final["OP"]=="MO12004") ].sort_values(by=["centro_produccion"], ascending=True).head(20)

,delay,display_name,fecha_final,cantidad_fabricada,id,OP,nombre,centro_produccion,maquina_real,nom_produccion,tiempo_real,maquina_esperada,tiempo_estandar_total,Clasificacion,tipo_union,desviacion_abs,precision_%
10444,9601.0,"ENTUBADORA 2 - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-26 06:54:41,700.0,95370,MO12004,FRANCY CAROLINA PINZON CHACON,CORTE,ENTUBADORA 2,DA2547,13.72,ENTUBADORA 2,40.0,AAA,exacta,-26.28,34.300000
97313,16016.0,"ENTUBADORA 1 - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-26 07:10:21,700.0,95354,MO12004,BLANCA DORA CUBIDES SANCHEZ,CORTE,ENTUBADORA 1,DA2547,22.88,ENTUBADORA 3,50.0,AAA,por_centro,-27.12,45.760000
97315,11746.0,"ENTUBADORA 1 - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-26 07:19:31,700.0,95356,MO12004,JAMER PUPO SIERRA,CORTE,ENTUBADORA 1,DA2547,16.78,ENTUBADORA 3,50.0,AAA,por_centro,-33.22,33.560000
10488,6871.0,"CONTROL DE CALIDAD - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-28 14:12:06,700.0,95726,MO12004,YENNY KATERINE PATAQUIVA NEUSA,EMBALAJE,CONTROL DE CALIDAD,DA2547,9.82,CONTROL DE CALIDAD,30.0,AAA,exacta,-20.18,32.733333
10490,5060.0,"EMBALAJE - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-28 14:16:56,700.0,95731,MO12004,INGRID CAROLINA GOMEZ DELPRADO,EMBALAJE,EMBALAJE,DA2547,7.23,EMBALAJE,45.0,AAA,exacta,-37.77,16.066667
10491,4434.0,"EMBALAJE - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-28 14:18:04,700.0,95732,MO12004,ESTEFANNY ALVAREZ VILLANUEVA,EMBALAJE,EMBALAJE,DA2547,6.33,EMBALAJE,45.0,AAA,exacta,-38.67,14.066667
10477,16198.0,"MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-28 14:11:11,700.0,95694,MO12004,ERIKA YULIEDT UMAÑA SABOGAL,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,DA2547,23.14,MARCADO/HOTMELT/ARANDELA/EMPAQUE,40.0,AAA,exacta,-16.86,57.850000
10487,8305.0,"CONTROL DE CALIDAD - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-28 14:11:38,700.0,95723,MO12004,MARIA HELENA DELPRADO PEREZ,EMBALAJE,CONTROL DE CALIDAD,DA2547,11.86,CONTROL DE CALIDAD,30.0,AAA,exacta,-18.14,39.533333
97585,23265.0,"INYECCION - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-28 13:21:34,700.0,95654,MO12004,YEISON VILLANUEVA MELO,INYECCION,INYECCION,DA2547,33.24,HENNECKE,225.0,AAA,por_centro,-191.76,14.773333
97322,23838.0,"INYECCION - DA2547 FILTROAIRE 1_ DONSSON -KODIAK, FORD 7000 CUM.,IHC 444E",2025-02-26 10:14:55,700.0,95365,MO12004,DICZON CAMILO REY RUIZ,INYECCION,INYECCION,DA2547,34.05,HENNECKE,225.0,AAA,por_centro,-190.95,15.133333


In [11]:
# --- Agrupar por los campos que identifican el mismo trabajo ---
cols_agrupacion = [
    "OP", "nombre", "centro_produccion", "maquina_real", "nom_produccion",
    "tiempo_estandar_total", "Clasificacion", "tipo_union", "maquina_esperada"
]

# Verificar que exista fecha_final
if "fecha_final" not in df_final.columns:
    df_final = df_final.rename(columns={"date_end": "fecha_final"})
    if "fecha_final" not in df_final.columns:
        df_final["fecha_final"] = pd.NaT

# Agrupación
df_join_sum = (
    df_final.groupby(cols_agrupacion, as_index=False)
    .agg({
        "tiempo_real": "sum",
        "cantidad_fabricada": "mean",
        "fecha_final": "max",
        "delay": "sum"
    })
)

# Asegurar que esté la columna
if "fecha_final" not in df_join_sum.columns:
    df_join_sum["fecha_final"] = pd.NaT

# Reordenar columnas
df_join_sum = df_join_sum[[
    "OP","nom_produccion","centro_produccion","nombre",
    "tiempo_estandar_total","maquina_real","tiempo_real",
    "maquina_esperada","delay","cantidad_fabricada","fecha_final","nombre","Clasificacion"
]]



In [12]:
df_join_sum[(df_join_sum["nom_produccion"]=="DA2547") & (df_join_sum["OP"]=="MO12552") ].sort_values(by=["centro_produccion","maquina_real"], ascending=True)

,OP,nom_produccion,centro_produccion,nombre,tiempo_estandar_total,maquina_real,tiempo_real,maquina_esperada,delay,cantidad_fabricada,fecha_final,nombre,Clasificacion
23808,MO12552,DA2547,CORTE,BLANCA DORA CUBIDES SANCHEZ,50.0,ENTUBADORA 3,12.55,ENTUBADORA 3,7528.0,600.0,2025-06-06 13:13:12,BLANCA DORA CUBIDES SANCHEZ,AAA
23817,MO12552,DA2547,CORTE,FRANCY CAROLINA PINZON CHACON,50.0,ENTUBADORA 3,17.20,ENTUBADORA 3,10318.0,600.0,2025-06-06 12:31:06,FRANCY CAROLINA PINZON CHACON,AAA
23820,MO12552,DA2547,CORTE,JAMER PUPO SIERRA,50.0,ENTUBADORA 3,23.63,ENTUBADORA 3,14179.0,600.0,2025-06-06 13:18:27,JAMER PUPO SIERRA,AAA
23818,MO12552,DA2547,EMBALAJE,INGRID CAROLINA GOMEZ DELPRADO,0.0,BOLSA,2.72,BOLSA,1631.0,600.0,2025-06-10 07:39:53,INGRID CAROLINA GOMEZ DELPRADO,AAA
23821,MO12552,DA2547,EMBALAJE,MARIA HELENA DELPRADO PEREZ,30.0,CONTROL DE CALIDAD,27.34,CONTROL DE CALIDAD,16403.0,600.0,2025-06-10 14:09:45,MARIA HELENA DELPRADO PEREZ,AAA
23825,MO12552,DA2547,EMBALAJE,YENNY KATERINE PATAQUIVA NEUSA,30.0,CONTROL DE CALIDAD,21.38,CONTROL DE CALIDAD,12826.0,600.0,2025-06-10 12:11:00,YENNY KATERINE PATAQUIVA NEUSA,AAA
23811,MO12552,DA2547,EMBALAJE,DEICY MILENA LOPEZ VELASQUEZ,45.0,EMBALAJE,20.65,EMBALAJE,12389.0,600.0,2025-06-11 07:18:03,DEICY MILENA LOPEZ VELASQUEZ,AAA
23814,MO12552,DA2547,EMBALAJE,ERIKA YULIEDT UMAÑA SABOGAL,45.0,EMBALAJE,4.79,EMBALAJE,2874.0,600.0,2025-06-10 14:07:14,ERIKA YULIEDT UMAÑA SABOGAL,AAA
23819,MO12552,DA2547,EMBALAJE,INGRID CAROLINA GOMEZ DELPRADO,45.0,EMBALAJE,0.54,EMBALAJE,323.0,600.0,2025-06-11 07:16:29,INGRID CAROLINA GOMEZ DELPRADO,AAA
23826,MO12552,DA2547,EMBALAJE,YENNY KATERINE PATAQUIVA NEUSA,45.0,EMBALAJE,5.88,EMBALAJE,3531.0,600.0,2025-06-10 14:17:37,YENNY KATERINE PATAQUIVA NEUSA,AAA


In [13]:
df_join_sum.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/analisis_personas_tiempos_op.xlsx")

In [14]:
df_join_sum = df_join_sum.loc[:, ~df_join_sum.columns.duplicated()]

# 🟩 1) Agrupación base (igual que antes)
df_producto_centro_maquina = (
    df_join_sum.groupby(
        ["OP","nom_produccion", "centro_produccion", "maquina_real", "maquina_esperada"],
        as_index=False
    )
    .agg({
        "tiempo_estandar_total": "mean",
        "tiempo_real": "sum",
        "delay": "sum",
        "cantidad_fabricada": "mean",
        "fecha_final": "max",
        "nombre":"nunique"
    }) .rename(columns={"nombre": "n_empleados"})
)

# 🟨 2) Caso especial DA2020 (XPEC + TERMOSELLADO)
mask_da2020_plisado = (
    df_producto_centro_maquina["nom_produccion"].astype(str).str.upper().str.contains("DA2020", na=False)
) & (df_producto_centro_maquina["centro_produccion"].astype(str).str.upper() == "PLISADO")

df_special = df_producto_centro_maquina[mask_da2020_plisado].copy()
df_rest = df_producto_centro_maquina[~mask_da2020_plisado].copy()

def has_both_xpec_termo(g):
    ms = [str(m).upper() for m in g["maquina_real"].fillna("")]
    return any("XPEC" in m for m in ms) and any("TERMOSELLADO" in m for m in ms)

ops_with_both = [op for op, g in df_special.groupby("OP") if has_both_xpec_termo(g)]

to_merge = df_special[df_special["OP"].isin(ops_with_both)].copy()
not_merged_special = df_special[~df_special["OP"].isin(ops_with_both)].copy()

if not to_merge.empty:
    merged_special = (
        to_merge.groupby(
            ["OP", "nom_produccion", "centro_produccion"], as_index=False
        )
        .agg({
            "tiempo_estandar_total": "mean",
            "tiempo_real": "sum",
            "delay": "sum",
            "cantidad_fabricada": "mean",
            "fecha_final": "max",
            "maquina_real": lambda s: "XPEC + TERMOSELLADO",
            "maquina_esperada": "first"
        })
    )
else:
    merged_special = pd.DataFrame(columns=df_special.columns)

df_producto_centro_maquina = pd.concat([df_rest, not_merged_special, merged_special], ignore_index=True)


# 🟧 3) NUEVO AJUSTE: solo dividir tiempo estándar en PLISADO si hay más de una máquina_real
mask_plisado = df_producto_centro_maquina["centro_produccion"].str.upper() == "PLISADO"

# Extraer solo filas de PLISADO
df_plisado = df_producto_centro_maquina[mask_plisado].copy()

# Calcular cuántas máquinas hay por OP solo dentro de PLISADO
plisado_counts = (
    df_plisado.groupby("OP")["maquina_real"]
    .nunique()
    .rename("num_maquinas")
    .reset_index()
)

# Unir ese conteo solo con las filas de PLISADO
df_plisado = df_plisado.merge(plisado_counts, on="OP", how="left")

# Dividir tiempo_estandar_total solo donde hay más de una máquina
df_plisado.loc[df_plisado["num_maquinas"] > 1, "tiempo_estandar_total"] = (
    df_plisado.loc[df_plisado["num_maquinas"] > 1, "tiempo_estandar_total"] /
    df_plisado.loc[df_plisado["num_maquinas"] > 1, "num_maquinas"]
)

# Quitar columna auxiliar
df_plisado.drop(columns="num_maquinas", inplace=True)

# Reemplazar en el DataFrame principal SOLO las filas de PLISADO
df_producto_centro_maquina = pd.concat(
    [df_producto_centro_maquina[~mask_plisado], df_plisado],
    ignore_index=True
)


# 🟦 4) Calcular fecha de cierre
fechas_cierre = (
    df_producto_centro_maquina.groupby("OP", as_index=False)["fecha_final"]
    .max()
    .rename(columns={"fecha_final": "fecha_cierre_op"})
)

df_producto_centro_maquina = pd.merge(
    df_producto_centro_maquina,
    fechas_cierre,
    on="OP",
    how="left"
)


# Asegurar que las columnas de tiempo sean numéricas y manejar NaNs (muy importante)
# Asume que tu DataFrame es 'df_producto_centro_maquina'
df_producto_centro_maquina['tiempo_estandar_total'] = pd.to_numeric(df_producto_centro_maquina['tiempo_estandar_total'], errors='coerce').fillna(0)
df_producto_centro_maquina['tiempo_real'] = pd.to_numeric(df_producto_centro_maquina['tiempo_real'], errors='coerce').fillna(0)



# Indicador 1: Diferencia de Tiempo (real vs. estándar)
# Valores POSITIVOS indican ineficiencia (se tardó más)
df_producto_centro_maquina.loc[:, 'Diferencia_Tiempo'] = df_producto_centro_maquina['tiempo_real'] - df_producto_centro_maquina['tiempo_estandar_total']

# Indicador 2: Desviación Absoluta (solo el tiempo perdido, donde real > estándar)
# NOTA: Este indicador captura SOLO la desviación negativa (tiempo extra consumido)
df_producto_centro_maquina.loc[:, 'Desviacion_Absoluta'] = np.maximum(0, df_producto_centro_maquina['Diferencia_Tiempo'])

# Indicador 2 Variación Porcentual (%) y Absoluta
# Cuánto más o menos tiempo se tardó con respecto al estándar.
df_producto_centro_maquina.loc[:, 'Variacion_Porcentual'] = np.where(
    df_producto_centro_maquina['tiempo_estandar_total'] != 0, 
    df_producto_centro_maquina['Diferencia_Tiempo'] / df_producto_centro_maquina['tiempo_estandar_total'], 
    np.nan
)


# Variación absoluta (en valor absoluto)
df_producto_centro_maquina['Variacion_Absoluta'] = np.abs(
    df_producto_centro_maquina['tiempo_real'] - df_producto_centro_maquina['tiempo_estandar_total']
)

In [15]:
df_producto_centro_maquina[(df_producto_centro_maquina["nom_produccion"]=="DA2547") & (df_producto_centro_maquina["OP"]=="MO12552") ].sort_values(by=["centro_produccion","maquina_real"], ascending=True)

,OP,nom_produccion,centro_produccion,maquina_real,maquina_esperada,tiempo_estandar_total,tiempo_real,delay,cantidad_fabricada,fecha_final,n_empleados,fecha_cierre_op,Diferencia_Tiempo,Desviacion_Absoluta,Variacion_Porcentual,Variacion_Absoluta
11889,MO12552,DA2547,CORTE,ENTUBADORA 3,ENTUBADORA 3,50.0,53.38,32025.0,600.0,2025-06-06 13:18:27,3.0,2025-06-11 07:18:15,3.38,3.38,0.067600,3.38
11890,MO12552,DA2547,EMBALAJE,BOLSA,BOLSA,0.0,2.72,1631.0,600.0,2025-06-10 07:39:53,1.0,2025-06-11 07:18:15,2.72,2.72,NaN,2.72
11891,MO12552,DA2547,EMBALAJE,CONTROL DE CALIDAD,CONTROL DE CALIDAD,30.0,48.72,29229.0,600.0,2025-06-10 14:09:45,2.0,2025-06-11 07:18:15,18.72,18.72,0.624000,18.72
11892,MO12552,DA2547,EMBALAJE,EMBALAJE,EMBALAJE,45.0,31.86,19117.0,600.0,2025-06-11 07:18:03,4.0,2025-06-11 07:18:15,-13.14,0.00,-0.292000,13.14
11893,MO12552,DA2547,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,40.0,16.12,9675.0,600.0,2025-06-11 07:18:15,2.0,2025-06-11 07:18:15,-23.88,0.00,-0.597000,23.88
11894,MO12552,DA2547,INYECCION,HENNECKE,HENNECKE,225.0,113.45,68066.0,600.0,2025-06-09 12:44:21,3.0,2025-06-11 07:18:15,-111.55,0.00,-0.495778,111.55
14991,MO12552,DA2547,PLISADO,PLISADORA 3,PLISADORA 4,85.0,67.36,40418.0,600.0,2025-06-07 08:21:34,2.0,2025-06-11 07:18:15,-17.64,0.00,-0.207529,17.64
14992,MO12552,DA2547,PLISADO,PLISADORA 4,PLISADORA 4,85.0,30.99,18596.0,600.0,2025-06-07 07:27:53,2.0,2025-06-11 07:18:15,-54.01,0.00,-0.635412,54.01


In [16]:
df_producto_centro_maquina.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/analisis_op_tiempos.xlsx")

In [17]:
df_final["nom_produccion"].unique()

array(['DA2020', 'DA4570', 'DA2666', 'DA2772', 'DA4570A', 'DA2690',
       'DA2570', 'DA2738', 'DA4982', 'DA4690', 'DA3066', 'DA8168',
       'DA4671', 'DA2968', 'DA2524', 'DA2957', 'DA2982', 'DA4783',
       'DA4666', 'DA4772', 'DA2547', 'DA2702', 'DA2671', 'DA2612',
       'DA2783', 'DA8108', 'DA2176', 'DA2926', 'DA2788', 'DA2937',
       'DA2902', 'DA4772A', 'DA2804', 'DA8264', 'DA9264', 'DA8218',
       'DA7802', 'DA2946', 'DA8154', 'DA4932', 'DA2109', 'DA4025',
       'DA4738', 'DA2304', 'DA8190', 'DA8057', 'DA4132', 'DA2639',
       'DA2988', 'DA8003', 'DA4937', 'DA2886', 'DA8072', 'DA4547',
       'DA4666A', 'DA2695', 'DA8234', 'DA2932', 'DA7824', 'DA9027',
       'DA9031', 'DA8184', 'DA2598', 'DA2416', 'DA2752', 'DA8220',
       'DA8076', 'DA8250', 'DA2873', 'DA2594', 'DA3370', 'DA8196',
       'DA8151', 'DA7817', 'DA2047', 'DA2346', 'DA7801', 'DA4869',
       'DA2106', 'DA4190', 'DA4265', 'DA2028', 'DA2869', 'DA2049',
       'DA2242', 'DA2546', 'DA2270', 'DA2881', 'DA8065', 'D